### 0. импорт библиотек

In [4]:
import sqlalchemy as sa
from sqlalchemy.orm import scoped_session, sessionmaker, declarative_base
from sqlalchemy.dialects.postgresql import UUID
from uuid import uuid4
import redis
from typing import Optional
from datetime import datetime, timedelta
from werkzeug.security import check_password_hash as _check_password_hash
from werkzeug.security import generate_password_hash as _generate_password_hash
import json
from jsonschema import validate
import numpy as np
import random
import string
import tqdm
import pandas as pd
import gc
from elasticsearch import Elasticsearch

In [3]:
#pip install tqdm


Note: you may need to restart the kernel to use updated packages.


### 1. базы данных

* postgres

In [8]:
engine = sa.create_engine(
    "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        user='app',
        password='123qwe',
        host='localhost',
        port=5432,
        database='user_database',
    )
)
db_session = scoped_session(
    sessionmaker(autocommit=False, autoflush=False, bind=engine)
)

Base = declarative_base()
Base.query = db_session.query_property()

* redis

In [10]:
cache = redis.StrictRedis(
    host='localhost',
    port=6379,
    db=0,
)

* elasticsearch

In [11]:
es_client = Elasticsearch(retry_on_timeout=True, host='localhost', port=9200)
es_client.indices.create(
    index='filmworks', 
    body={
        "settings": {
            "refresh_interval": "1s",
            "analysis": {
                "filter": {
                    "english_stop": {
                        "type": "stop",
                        "stopwords": "_english_"
                    },
                    "english_stemmer": {
                        "type": "stemmer",
                        "language": "english"
                    },
                    "english_possessive_stemmer": {
                        "type": "stemmer",
                        "language": "possessive_english"
                    },
                    "russian_stop": {
                        "type": "stop",
                        "stopwords": "_russian_"
                    },
                    "russian_stemmer": {
                        "type": "stemmer",
                        "language": "russian"
                    }
                },
                "analyzer": {
                    "ru_en": {
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "english_stop",
                            "english_stemmer",
                            "english_possessive_stemmer",
                            "russian_stop",
                            "russian_stemmer"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "true",
            "properties": {
                "id": {
                    "type": "keyword"
                },
                "title": {
                    "type": "text",
                    "analyzer": "ru_en",
                    "fields": {
                        "raw": {
                            "type": "keyword"
                        }
                    }
                },
                "description": {
                    "type": "text",
                    "analyzer": "ru_en"
                },
                "creation_date": {
                    "type": "date"
                },
                "imdb_rating": {
                    "type": "float"
                },
                "type": {
                    "type": "text"
                },
                "access_level": {
                    "type": "integer"
                },
                "file_path": {
                    "type": "text"
                },
                "updated_at": {
                    "type": "date"
                },
                "genres__name": {
                    "type": "text"
                },
                "directors__full_name": {
                    "type": "text",
                    "analyzer": "ru_en"
                },
                "actors__full_name": {
                    "type": "text",
                    "analyzer": "ru_en"
                },
                "writers__full_name": {
                    "type": "text",
                    "analyzer": "ru_en"
                },
                "genres": {
                    "type": "nested",
                    "dynamic": "true",
                    "properties": {
                        "id": {
                            "type": "keyword"
                        },
                        "name": {
                            "type": "text",
                            "analyzer": "ru_en"
                        },
                        "description": {
                            "type": "text",
                            "analyzer": "ru_en"
                        }
                    }
                },
                "directors": {
                    "type": "nested",
                    "dynamic": "true",
                    "properties": {
                        "id": {
                            "type": "keyword"
                        },
                        "full_name": {
                            "type": "text",
                            "analyzer": "ru_en"
                        }
                    }
                },
                "actors": {
                    "type": "nested",
                    "dynamic": "true",
                    "properties": {
                        "id": {
                            "type": "keyword"
                        },
                        "full_name": {
                            "type": "text",
                            "analyzer": "ru_en"
                        }
                    }
                },
                "writers": {
                    "type": "nested",
                    "dynamic": "true",
                    "properties": {
                        "id": {
                            "type": "keyword"
                        },
                        "full_name": {
                            "type": "text",
                            "analyzer": "ru_en"
                        }
                    }
                }
            }
        }
    }
)

es_client.indices.create(
    index='genres', 
    body={        
        "settings": {
            "refresh_interval": "1s",
            "analysis": {
                "filter": {
                    "english_stop": {
                        "type": "stop",
                        "stopwords": "_english_"
                    },
                    "english_stemmer": {
                        "type": "stemmer",
                        "language": "english"
                    },
                    "english_possessive_stemmer": {
                        "type": "stemmer",
                        "language": "possessive_english"
                    },
                    "russian_stop": {
                        "type": "stop",
                        "stopwords": "_russian_"
                    },
                    "russian_stemmer": {
                        "type": "stemmer",
                        "language": "russian"
                    }
                },
                "analyzer": {
                    "ru_en": {
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "english_stop",
                            "english_stemmer",
                            "english_possessive_stemmer",
                            "russian_stop",
                            "russian_stemmer"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "true",
            "properties": {
                "id": {
                    "type": "keyword"
                },
                "name": {
                    "type": "text"
                },
                "description": {
                    "type": "text"
                },
                "filmworks__id": {
                    "type": "keyword"
                }
            }
        }
    }
)

es_client.indices.create(
    index='actors', 
    body={        
        "settings": {
            "refresh_interval": "1s",
            "analysis": {
                "filter": {
                    "english_stop": {
                        "type": "stop",
                        "stopwords": "_english_"
                    },
                    "english_stemmer": {
                        "type": "stemmer",
                        "language": "english"
                    },
                    "english_possessive_stemmer": {
                        "type": "stemmer",
                        "language": "possessive_english"
                    },
                    "russian_stop": {
                        "type": "stop",
                        "stopwords": "_russian_"
                    },
                    "russian_stemmer": {
                        "type": "stemmer",
                        "language": "russian"
                    }
                },
                "analyzer": {
                    "ru_en": {
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "english_stop",
                            "english_stemmer",
                            "english_possessive_stemmer",
                            "russian_stop",
                            "russian_stemmer"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "true",
            "properties": {
                "id": {
                    "type": "keyword"
                },
                "full_name": {
                    "type": "text"
                },                
                "filmworks__id": {
                    "type": "keyword"
                }
            }
        }
    }
)
es_client.indices.create(
    index='writers', 
    body={        
        "settings": {
            "refresh_interval": "1s",
            "analysis": {
                "filter": {
                    "english_stop": {
                        "type": "stop",
                        "stopwords": "_english_"
                    },
                    "english_stemmer": {
                        "type": "stemmer",
                        "language": "english"
                    },
                    "english_possessive_stemmer": {
                        "type": "stemmer",
                        "language": "possessive_english"
                    },
                    "russian_stop": {
                        "type": "stop",
                        "stopwords": "_russian_"
                    },
                    "russian_stemmer": {
                        "type": "stemmer",
                        "language": "russian"
                    }
                },
                "analyzer": {
                    "ru_en": {
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "english_stop",
                            "english_stemmer",
                            "english_possessive_stemmer",
                            "russian_stop",
                            "russian_stemmer"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "true",
            "properties": {
                "id": {
                    "type": "keyword"
                },
                "full_name": {
                    "type": "text"
                },                
                "filmworks__id": {
                    "type": "keyword"
                }
            }
        }
    }
)
es_client.indices.create(
    index='directors', 
    body={        
        "settings": {
            "refresh_interval": "1s",
            "analysis": {
                "filter": {
                    "english_stop": {
                        "type": "stop",
                        "stopwords": "_english_"
                    },
                    "english_stemmer": {
                        "type": "stemmer",
                        "language": "english"
                    },
                    "english_possessive_stemmer": {
                        "type": "stemmer",
                        "language": "possessive_english"
                    },
                    "russian_stop": {
                        "type": "stop",
                        "stopwords": "_russian_"
                    },
                    "russian_stemmer": {
                        "type": "stemmer",
                        "language": "russian"
                    }
                },
                "analyzer": {
                    "ru_en": {
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "english_stop",
                            "english_stemmer",
                            "english_possessive_stemmer",
                            "russian_stop",
                            "russian_stemmer"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "true",
            "properties": {
                "id": {
                    "type": "keyword"
                },
                "full_name": {
                    "type": "text"
                },                
                "filmworks__id": {
                    "type": "keyword"
                }
            }
        }
    }
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'directors'}

### 2. модели

##### 2.1. определение

In [12]:
def generate_now():
    return datetime.utcnow()

class DatetimeMixin(object):
    created_at = sa.Column(
        sa.DateTime, unique=False, nullable=False, default=generate_now()
    )
    updated_at = sa.Column(sa.DateTime, unique=False, nullable=True) 

def generate_uuid4():
    return uuid4()

class UUIDMixin(object):
    id = sa.Column(UUID(as_uuid=True), primary_key=True, default=generate_uuid4())    


class User(UUIDMixin, DatetimeMixin, Base):
    __tablename__ = "user"

    # поля
    login = sa.Column(sa.String, unique=True, nullable=False)
    password = sa.Column(sa.String, unique=False, nullable=False)
    email = sa.Column(sa.Text, unique=True, nullable=False)
    city = sa.Column(sa.String, unique=False, nullable=True)

    # отношения
    sessions = sa.orm.relationship(
        "Session", back_populates="user", cascade="all,delete", passive_deletes=True
    )
    social_accounts = sa.orm.relationship(
        "SocialAccount",
        back_populates="user",
        cascade="all,delete",
        passive_deletes=True,
    )
    roles = sa.orm.relationship(
        "Role",
        secondary="user__role",
        back_populates="users",
        cascade="all, delete",
        passive_deletes=True,
    )

    def __init__(
        self, login: str, password: str, email: str, city: Optional[str] = None
    ):
        self.login = login
        self.password = password
        self.email = email
        self.city = city
        
    def generate_password_hash(self, password):
        self.password_hash = _generate_password_hash(password, "scrypt")

    def check_password_hash(self, password):
        return _check_password_hash(self.password_hash, password)

    @classmethod
    def get_user_by_login(
        cls, login: Optional[str] = None, email: Optional[str] = None
    ):
        return (
            db_session.query(cls)
            .filter(sa.or_(cls.login == login, cls.email == email))
            .first()
        )

    def __repr__(self):
        return f"<User {self.login}>"
    
class Role(UUIDMixin, DatetimeMixin, Base):
    __tablename__ = "role"

    # поля
    name = sa.Column(sa.String, unique=True, nullable=False)
    description = sa.Column(sa.String, unique=True, nullable=False)

    # отношения
    users = sa.orm.relationship(
        "User",
        secondary="user__role",
        back_populates="roles",
        cascade="all, delete",
        passive_deletes=True,
    )
    permissions = sa.orm.relationship(
        "Role",
        secondary="user__role",
        back_populates="roles",
        cascade="all, delete",
        passive_deletes=True,
    )

    def __init__(self, name: str, description: Optional[str] = None):
        self.name = name
        self.description = description

    def __repr__(self):
        return f"<Role {self.name}>"
    
user__role = sa.Table(
    "user__role",
    Base.metadata,
    sa.Column("user_id", sa.ForeignKey("user.id", ondelete="CASCADE")),
    sa.Column("role_id", sa.ForeignKey("role.id", ondelete="CASCADE")),
)

class RefreshToken(UUIDMixin, DatetimeMixin, Base):
    __tablename__ = "refresh_token"

    # поля
    session_id = sa.Column(
        UUID(as_uuid=True), sa.ForeignKey("user.id", ondelete="CASCADE")
    )
    token = sa.Column(sa.Text, unique=True, nullable=False)

    # отношения
    session = sa.orm.relationship(
        "Session", back_populates="refresh_tokens", cascade="all,delete"
    )

    def __init__(self, session_id: uuid4, token: str):
        self.session_id = session_id
        self.token = token

    def __repr__(self):
        return f"<Token {self.token}:{self.session_id}>"
    
YEARS = list[range(2000, 2026)]
MONTHS = list(range(1, 13))


def create_partition(target, connection, **kw) -> None:
    for year in YEARS:
        for month in MONTHS:
            patition_name = f"user_{year}_{month}"
            low, high = f"{year}-{month}-01", f"{year}-{month}-31"
            text_sql = f"""
                    CREATE TABLE IF NOT EXISTS "{patition_name}" 
                    PARTITION OF "session" FOR VALUES FROM ('{low}') TO ('{high}')"""
            text_sql = sa.text(text_sql)
            connection.execute(text_sql)


class Session(UUIDMixin, DatetimeMixin, Base):
    __tablename__ = "session"

    # поля
    user_id = sa.Column(
        UUID(as_uuid=True), sa.ForeignKey("user.id", ondelete="CASCADE")
    )
    user_agent = sa.Column(sa.Text, unique=False, nullable=False)
    created_at = sa.Column(sa.DateTime, default=generate_now(), primary_key=True)

    # отношения
    user = sa.orm.relationship("User", back_populates="sessions", cascade="all,delete")
    refresh_tokens = sa.orm.relationship(
        "Token", back_populates="session", cascade="all,delete", passive_deletes=True
    )

    def __init__(
        self,
        user_id: uuid4,
        user_agent: str,
    ):
        self.user_id = user_id
        self.user_agent = user_agent

    def __repr__(self):
        return f"<Session: {self.user_id}:{self.created_at }>"
    
class SocialAccount(UUIDMixin, DatetimeMixin, Base):
    __tablename__ = "social_account"
    __table_args__ = (sa.UniqueConstraint("social_id", "social_name", name="social_pk"),)

    # поля
    user_id = sa.Column(
        UUID(as_uuid=True), sa.ForeignKey("user.id", ondelete="CASCADE")
    )
    social_id = sa.Column(sa.Text, unique=False, nullable=False)    
    social_name = sa.Column(sa.String, unique=False, nullable=False)

    # отношения
    user = sa.orm.relationship(
        "User", back_populates="social_accounts", cascade="all,delete"
    )

    def __init__(
        self,
        name: str,
    ):
        self.name = name

    def __repr__(self):
        return f"<SocialAccount {self.name}:{self.user_id}>"

##### 2.1. создание таблиц

In [13]:
Base.metadata.create_all(bind=engine) 

for table in Base.metadata.tables:
    print(table)


user
role
user__role
refresh_token
session
social_account


##### 2.2. схемы для контрактов

In [14]:
postgers_user_schema = {
  "$schema": "http://json-schema.org/schema#",
  "title": "JSON schema for User data",
  "type": "object",
  "required": ['id', 'login', 'password', 'email', 'created_at', 'updated_at'],
  "properties": {
    "id": {"type": "string", "format": "uuid"},
    "login": {"type": "string" },
    "password": {"type": "string" },
    "email": {"type": "string", "pattern": "^.+@.+\\..+$"},
    "created_at": {"type": "string","format": "date-time"},
    "updated_at": {"type": "string","format": "date-time"},
  }
} 

postgres_role_schema = {
  "$schema": "http://json-schema.org/schema#",
  "title": "JSON schema for Role data",
  "type": "object",
  "required": ['id', 'name', 'created_at', 'updated_at'],
  "properties": {
    "id": {"type": "string", "format": "uuid"},
    "name": {"type": "string" },    
    "created_at": {"type": "string","format": "date-time"},
    "updated_at": {"type": "string","format": "date-time"},
  }
} 

postgres_session_schema = {
  "$schema": "http://json-schema.org/schema#",
  "title": "JSON schema for Session data",
  "type": "object",
  "required": ['id', 'user_id', 'user_agent', 'created_at', 'updated_at'],
  "properties": {
    "id": {"type": "string", "format": "uuid"},
    "user_id": {"type": "string", "format": "uuid"},
    "user_agent": {"type": "string" },    
    "created_at": {"type": "string","format": "date-time"},
    "updated_at": {"type": "string","format": "date-time"},
  }
} 

postgres_social_account_schema = {
  "$schema": "http://json-schema.org/schema#",
  "title": "JSON schema for SocialAccount data",
  "type": "object",
  "required": ['id', 'user_id', 'name', 'created_at', 'updated_at'],
  "properties": {
    "id": {"type": "string", "format": "uuid"},
    "user_id": {"type": "string", "format": "uuid"},
    "social_id":{"type": "string"}, 
    "social_name": {"type": "string"},    
    "created_at": {"type": "string","format": "date-time"},
    "updated_at": {"type": "string","format": "date-time"},
  }
} 


elastic_filmwork = {
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "id": {
      "type": "string"
    },
    "title": {
      "type": "string"
    },
    "description": {
      "type": "string"
    },
    "type": {
      "type": "string"
    },
    "creation_date": {
      "type": "string",
      "format": "date"
    },
    "file_path": {
      "type": "string"
    },
    "access_level": {
      "type": "integer"
    },
    "genres__name": {
      "type": "array",
      "items": [
        {
          "type": "string"
        }
      ]
    },
    "actors__name": {
      "type": "array",
      "items": [
        {
          "type": "string"
        }
      ]
    },
    "directors__name": {
      "type": "array",
      "items": [
        {
          "type": "string"
        }
      ]
    },
    "writers__name": {
      "type": "array",
      "items": [
        {
          "type": "string"
        }
      ]
    },
    "genres": {
      "type": "array",
      "items": [
        {
          "type": "object",
          "properties": {
            "id": {
              "type": "string"
            },
            "name": {
              "type": "string"
            }
          },
          "required": [
            "id",
            "name"
          ]
        }
      ]
    },
    "actors": {
      "type": "array",
      "items": [
        {
          "type": "object",
          "properties": {
            "id": {
              "type": "string"
            },
            "full_name": {
              "type": "string"
            }
          },
          "required": [
            "id",
            "full_name"
          ]
        }
      ]
    },
    "writers": {
      "type": "array",
      "items": [
        {
          "type": "object",
          "properties": {
            "id": {
              "type": "string"
            },
            "full_name": {
              "type": "string"
            }
          },
          "required": [
            "id",
            "full_name"
          ]
        }
      ]
    },
    "directors": {
      "type": "array",
      "items": [
        {
          "type": "object",
          "properties": {
            "id": {
              "type": "string"
            },
            "full_name": {
              "type": "string"
            }
          },
          "required": [
            "id",
            "full_name"
          ]
        }
      ]
    }
  },
  "required": [
    "id",
    "title",
    "description",
    "type",
    "creation_date",
    "file_path",
    "access_level",
    "genres__name",
    "actors__name",
    "directors__name",
    "writers__name",
    "genres",
    "actors",
    "writers",
    "directors"
  ]
}

elastic_genre = {
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "id": {
      "type": "string"
    },
    "name": {
      "type": "string"
    },
    "description": {
      "type": "string"
    },
    "filmworks__id": {
      "type": "array",
      "items": [
        {
          "type": "string"
        }        
      ]
    }
  },
  "required": [
    "id",
    "name",
    "filmworks__id"
  ]
}


elastic_actor = {
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "id": {
      "type": "string"
    },
    "full_name": {
      "type": "string"
    },
    "filmworks__id": {
      "type": "array",
      "items": [
        {
          "type": "string"
        },        
      ]
    }
  },
  "required": [
    "id",
    "full_name",
    "filmworks__id"
  ]
}


elastic_writer = {
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "id": {
      "type": "string"
    },
    "full_name": {
      "type": "string"
    },
    "filmworks__id": {
      "type": "array",
      "items": [
        {
          "type": "string"
        }        
      ]
    }
  },
  "required": [
    "id",
    "full_name",
    "filmworks__id"
  ]
}

elastic_director = {
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "id": {
      "type": "string"
    },
    "full_name": {
      "type": "string"
    },
    "filmworks__id": {
      "type": "array",
      "items": [
        {
          "type": "string"
        }        
      ]
    }
  },
  "required": [
    "id",
    "full_name",
    "filmworks__id"
  ]
}





In [15]:
d = {"id": uuid4().hex,
"title": 'title',
"description":'descr', 
"type": 'movie',  
"creation_year": ,
"file_path":'', 
"access_level": 1, 
"genres__name": ['g1'],
"actors__name": ['g1'],
"directors__name": ['g1'],
"writers__name": ['g1'],
"genres":[{"id":uuid4().hex, "name":'name1'}],
"actors":[{"id":uuid4().hex, "full_name":'name1'}],
"writers":[{"id":uuid4().hex, "full_name":'name1'}],
"directors":[{"id":uuid4().hex, "full_name":'name1'}]
}

SyntaxError: expression expected after dictionary key and ':' (331527674.py, line 5)

In [16]:
# 
d = {"id":uuid4().hex, "name":'name1', 'filmworks__id':[uuid4().hex, uuid4().hex, uuid4().hex]}
d

{'id': '5de55657bc9344e39c527f555d47bd95',
 'name': 'name1',
 'filmworks__id': ['2dfb2666792846f2846c286cd6a16fec',
  '5bbfa2613a634b358f4dbbfd2db0b962',
  'd13dd1208af84902ac5c0dce35c24877']}

##### 2.3. генерирование тестовых данных

In [17]:
def generate_test_data(user_count:int, session_count:int, chunk_size:int, seed:int) -> None:
    
    LETTERS = list(string.ascii_lowercase)
    SYMBOLS = list(string.punctuation)    

    user_data = []
    for _ in tqdm.tqdm(range(user_count)):
        np.random.seed(seed+_)
        random.seed(seed+_)
        
        id = uuid4()
        login = ''.join(np.random.choice(LETTERS, 10))
        email  = f'{login}@example.com'
        password = ''.join([''.join(np.random.choice(LETTERS, 5)), ''.join(np.random.choice(SYMBOLS, 5))])
        created_at = datetime.utcnow().isoformat()
        
        row = {
            'id': id.hex, 
            'login': login,     
            'email': email, 
            'password':password, 
            'created_at':created_at
        }    
        
        user_data.append(row)        
    df_user = pd.DataFrame.from_records(user_data)
    df_user.to_sql('user', con=engine, chunksize=chunk_size, if_exists='append', index=False)

    role_data = []
    role_data.append(
        {
            'id': uuid4().hex, 
            'name': 'суперпользователь',
            'description':'разрешено: все', 
            'created_at':datetime.utcnow().isoformat()
        }    
    ) 
    role_data.append(
        {
            'id': uuid4().hex,
            'name': 'пользователь',
            'description':'разрешено: регистрация, вход в аккаунт, выход из аккаунта, обновление токена', 
            'created_at':datetime.utcnow().isoformat()
        }    
    ) 
    role_data.append(
        {
            'id': uuid4().hex,
            'name': 'аналитик',
            'description':'разрешено: чтение сессий пользователя', 
            'created_at':datetime.utcnow().isoformat()
        }    
    )
    role_data.append(
        {
            'id': uuid4().hex,
            'name': 'гость',
            'description':'разрешено: вход в аккаунт', 
            'created_at':datetime.utcnow().isoformat()
        }    
    )        
    df_role = pd.DataFrame.from_records(role_data)
    df_role.to_sql('role', con=engine, chunksize=chunk_size, if_exists='append', index=False)

    user__role = []
    for user_id in tqdm.tqdm(df_user['id']):
        role_id = np.random.choice(df_role['id'].iloc[1:].values, p=[.89, .01, .1])    
        row = {
            'user_id':user_id,
            'role_id':role_id
        }
        user__role.append(row)        
    df_user__role = pd.DataFrame.from_records(user__role)
    df_user__role.to_sql('user__role', con=engine, chunksize=chunk_size, if_exists='append', index=False)

    ids = df_user['id'].values
    total = len(ids)
    chunk_size=10_000
    chunks = np.array_split(ids, np.ceil(total/chunk_size))
    for chunk in tqdm.tqdm(chunks):
        sessions = []
        for user_id in chunk:   
            sessions = []  
            now = datetime.utcnow()
            for _ in range(session_count):
                row = {
                    'id' : uuid4().hex,
                    'user_id':user_id,
                    'user_agent':'user_agent',
                    'created_at':now.isoformat(),
                    'updated_at':(now+timedelta(minutes=np.random.lognormal(10, 1))).isoformat()
                }
                sessions.append(row)
        df_sessions = pd.DataFrame.from_records(sessions)
        df_sessions.to_sql('session', con=engine, chunksize=chunk_size, if_exists='append', index=False) 
        del df_sessions
        gc.collect()

    social_accounts = []  
    
    for user_id in tqdm.tqdm(df_user['id']):     
        row = {
            'id' : uuid4().hex,
            'user_id':user_id,
            'social_id':uuid4().hex,
            'social_name':'vk',
            'created_at':datetime.utcnow().isoformat(),            
        }
        social_accounts.append(row)
    df_social_accounts = pd.DataFrame.from_records(social_accounts)
    df_social_accounts.to_sql('social_account', con=engine, chunksize=chunk_size, if_exists='append', index=False) 
    
    del df_user, df_role, df_user__role, df_social_accounts
    gc.collect() 

In [18]:
# заполняем бд пользователей тестовыми данными
generate_test_data(
    user_count=1_000, 
    session_count=100, 
    chunk_size=10_000, 
    seed=13
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 73259.11it/s]
